In [7]:
import pandas as pd

In [ ]:
### TODO
### import all prodigy functions
### figure out how to process information wthin the string into prodigy for annotation
### determine whether reading dataframe into prodigy is possible

In [9]:
file_path = "./data/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/"

In [11]:
df = pd.read_csv(file_path+"discharge.csv")

In [12]:
print(df.head())

          note_id  subject_id   hadm_id note_type  note_seq   
0  10000032-DS-21    10000032  22595853        DS        21  \
1  10000032-DS-22    10000032  22841357        DS        22   
2  10000032-DS-23    10000032  29079034        DS        23   
3  10000032-DS-24    10000032  25742920        DS        24   
4  10000084-DS-17    10000084  23052089        DS        17   

             charttime            storetime   
0  2180-05-07 00:00:00  2180-05-09 15:26:00  \
1  2180-06-27 00:00:00  2180-07-01 10:15:00   
2  2180-07-25 00:00:00  2180-07-25 21:42:00   
3  2180-08-07 00:00:00  2180-08-10 05:43:00   
4  2160-11-25 00:00:00  2160-11-25 15:09:00   

                                                text  
0   \nName:  ___                     Unit No:   _...  
1   \nName:  ___                     Unit No:   _...  
2   \nName:  ___                     Unit No:   _...  
3   \nName:  ___                     Unit No:   _...  
4   \nName:  ___                    Unit No:   __...  


In [17]:
test = df["text"][0]

In [18]:
with open('medical_report.txt', 'w') as f:
    f.write(test)